In [1]:
#Importing libraries
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, SimpleRNN, Embedding
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.optimizers import SGD

import tensorflow as tf
import pandas as pd
import tokenize, glob
import os, re, requests, tarfile, shutil, json, glob # For file manipualation only.


/Users/jahnavinp/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%tensorflow_version X.X
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
# Experiment with tokenize library to see its use
with tokenize.open('Data/p00002/Python/s005515634.py') as f:
    tokens = tokenize.generate_tokens(f.readline)
    for token in tokens:
        print(token)

TokenInfo(type=1 (NAME), string='while', start=(1, 0), end=(1, 5), line='while True:\n')
TokenInfo(type=1 (NAME), string='True', start=(1, 6), end=(1, 10), line='while True:\n')
TokenInfo(type=54 (OP), string=':', start=(1, 10), end=(1, 11), line='while True:\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(1, 11), end=(1, 12), line='while True:\n')
TokenInfo(type=5 (INDENT), string='    ', start=(2, 0), end=(2, 4), line='    try:\n')
TokenInfo(type=1 (NAME), string='try', start=(2, 4), end=(2, 7), line='    try:\n')
TokenInfo(type=54 (OP), string=':', start=(2, 7), end=(2, 8), line='    try:\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(2, 8), end=(2, 9), line='    try:\n')
TokenInfo(type=5 (INDENT), string='        ', start=(3, 0), end=(3, 8), line='        a, b = map(int, input().split())\n')
TokenInfo(type=1 (NAME), string='a', start=(3, 8), end=(3, 9), line='        a, b = map(int, input().split())\n')
TokenInfo(type=54 (OP), string=',', start=(3, 9), end=(3, 10), line=' 

In [4]:
all_files = glob.glob('data/*/Python/*.py')
print(len(all_files))
np.savetxt('all_py_files.txt', X = all_files, delimiter=',', comments='', fmt='%s')

1600


In [5]:
all_files_mini = glob.glob('data/*/Python/*.py')
np.savetxt('all_py_files.txt', X = all_files_mini, delimiter=',', comments='', fmt='%s')
len(all_files_mini)

1600

In [6]:
def tokenize_and_save(dir_name):
  tokenizerD = Tokenizer(
    num_words=10000,
    filters='',
    lower=False, split=' ', char_level=False, oov_token="<UNK>",
    document_count=0
  )
  
  # Assumes a file with all_py_files.txt name
  file_index = 1
  # with open(dir_name + 'all_py_files.txt') as dir_f:
  #   for py_file in dir_f.readlines():
  for py_file in all_files:
    file_index = file_index + 1
    with tokenize.open(py_file.strip()) as f:
      try:
        tokens = [tok.string for tok in tokenize.generate_tokens(f.readline)]
        tokenizerD.fit_on_texts(tokens)
      except:
        pass 

      if file_index % 2000 == 0:
        print(str(file_index) + " python files tokenized with #tokens: " + str(len(tokenizerD.word_index)))
        tokenizer_json = tokenizerD.to_json()
        with open(dir_name + 'tokenizer.json', 'w') as f:
          json.dump(tokenizer_json, f)

  # Save final version
  tokenizer_json = tokenizerD.to_json()
  with open(dir_name + 'tokenizer.json', 'w') as f:
    json.dump(tokenizer_json, f)

  print('Total tokenizer vocabulary {}'.format(len(tokenizerD.word_index)))


tokenize_and_save('IBM project')

Total tokenizer vocabulary 2091


In [7]:
#Load tokens into 'tokenizer' file. Total tokens created is 213461

path_full = "tokenizer.json"
# path_full = "drive/MyDrive/code_completion_data/samples/tokenizer.json"

with open(path_full) as f:
  json_string = json.load(f)

tokenizer = keras.preprocessing.text.tokenizer_from_json(
    json_string
)
len(tokenizer.word_index)

1048

In [8]:
with open('tokenizer.json') as f:
  json_string = json.load(f)

tokenizer_mini = keras.preprocessing.text.tokenizer_from_json(
    json_string
)

len(tokenizer_mini.word_index)

1048

In [9]:
# files is a list of python files to generate vector on
def vectorize_and_save(files):
  # Load tokenizer
  with open("tokenizer.json") as f:
    json_string = json.load(f)

  tokenizer = keras.preprocessing.text.tokenizer_from_json(
      json_string
  )
  
  seq_length = 10
  header_str = "X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y"
  file_index = 1
  py_file_index = 1
  dataX = []
  n_patterns = 0
  max_rows_in_each_file = 10000000
  for py_file in files:
    with tokenize.open(py_file.strip()) as f:
      try:
        py_file_index = py_file_index + 1
        py_tokens = [tok.string for tok in tokenize.generate_tokens(f.readline)]
        # Now convert the python tokens into an array of tokens from our Corpus tokenizer

        words_tokens = [0]*seq_length + [tokenizer.word_index[w] if w in tokenizer.word_index else tokenizer.word_index['<UNK>'] for w in py_tokens] # Get token id if exists else use <UNK>

        for j in range(len(words_tokens) - seq_length):
          dataX.append(words_tokens[j:j+seq_length+1])

        # Save to file if data reaches max count or if its the last row.
        if len(dataX) >= max_rows_in_each_file:
          file_name = 'dataxy_10_1_{}.csv'.format(file_index)
          np.savetxt(fname = file_name, 
                    X = dataX, 
                    delimiter=',', 
                    comments='',
                    fmt='%d',
                    header=header_str)
          n_patterns = n_patterns + len(dataX)
          dataX = []
          file_index = file_index + 1
          print("Saved to file " + file_name)

        if py_file_index % 2000 == 0:
          print("Processed files {} with dataset size {}".format(py_file_index, n_patterns + len(dataX)))
      except:
        pass 

  # Save once more for left items unsaved
  file_name = 'dataxy_10_1_{}.csv'.format(file_index)
  np.savetxt(fname = file_name, 
            X = dataX, 
            delimiter=',', 
            comments='',
            fmt='%d',
            header=header_str)
  n_patterns = n_patterns + len(dataX)
  
  print("Saved to file " + file_name)

  print ("Total Patterns: ", n_patterns)

In [10]:
py_files = glob.iglob('data/*/Python/*.py')
vectorize_and_save(py_files)

Saved to file dataxy_10_1_1.csv
Total Patterns:  130369


In [11]:
# Now generate on entire dataset full
py_files = glob.iglob('data/*/Python/*.py')
vectorize_and_save(py_files)

Saved to file dataxy_10_1_1.csv
Total Patterns:  130369


In [12]:
class CompletionDataGenerator(keras.utils.Sequence) :

  def __init__(self, files_list, batch_size) :
    self.files_list = pd.read_csv(files_list, header = 0)
    self.files_list['lines'] = self.files_list['lines'].astype('int')
    self.batch_size = batch_size
    self.total_count = self.files_list['lines'].sum()
    print('Loaded data generator with total lines {} and details:'.format(self.total_count))
    display(self.files_list)
    
    
  def __len__(self) :
    return (np.ceil(self.total_count / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return None

CompletionDataGenerator('lines_count.csv', 64)

FileNotFoundError: [Errno 2] No such file or directory: 'lines_count.csv'

In [13]:
dataxy = pd.read_csv('dataxy_10_1_1.csv', header = 0)
display(dataxy)
dataX = dataxy.iloc[:, 0:10].values
dataY = dataxy.iloc[:, 10].values

print(dataX)
print(dataY)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
0,0,0,0,0,0,0,0,0,0,0,27
1,0,0,0,0,0,0,0,0,0,27,5
2,0,0,0,0,0,0,0,0,27,5,45
3,0,0,0,0,0,0,0,27,5,45,3
4,0,0,0,0,0,0,27,5,45,3,1
...,...,...,...,...,...,...,...,...,...,...,...
130364,5,52,3,105,4,2,18,3,50,3,105
130365,52,3,105,4,2,18,3,50,3,105,4
130366,3,105,4,2,18,3,50,3,105,4,4
130367,105,4,2,18,3,50,3,105,4,4,1


[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0  27]
 [  0   0   0 ...   0  27   5]
 ...
 [  3 105   4 ...   3 105   4]
 [105   4   2 ... 105   4   4]
 [  4   2  18 ...   4   4   1]]
[27  5 45 ...  4  1  1]


In [14]:
tokenizer.word_index

{'<UNK>': 1,
 '\n': 2,
 '(': 3,
 ')': 4,
 '=': 5,
 ':': 6,
 ',': 7,
 '.': 8,
 '1': 9,
 '_': 10,
 'a': 11,
 '"': 12,
 '[': 13,
 ']': 14,
 'x': 15,
 '0': 16,
 'i': 17,
 'print': 18,
 'input': 19,
 '*': 20,
 'y': 21,
 'n': 22,
 'int': 23,
 '2': 24,
 "'": 25,
 'if': 26,
 's': 27,
 'in': 28,
 'for': 29,
 '-': 30,
 'b': 31,
 'w': 32,
 'h': 33,
 'ans': 34,
 '+': 35,
 'split': 36,
 '/': 37,
 'r': 38,
 'else': 39,
 'range': 40,
 'max': 41,
 'map': 42,
 '#': 43,
 '9': 44,
 'list': 45,
 '%': 46,
 '\t': 47,
 'import': 48,
 'math': 49,
 'len': 50,
 'l': 51,
 'str': 52,
 'pi': 53,
 'def': 54,
 'append': 55,
 'and': 56,
 'return': 57,
 'elif': 58,
 'sys': 59,
 '>': 60,
 'c': 61,
 '3': 62,
 '5': 63,
 '{': 64,
 '}': 65,
 'f': 66,
 '!': 67,
 'm': 68,
 'gcd': 69,
 'while': 70,
 '<': 71,
 '6': 72,
 'min': 73,
 'float': 74,
 'num': 75,
 'main': 76,
 'join': 77,
 "''": 78,
 '8': 79,
 'stdin': 80,
 'd': 81,
 'format': 82,
 't': 83,
 'true': 84,
 '?': 85,
 'or': 86,
 'from': 87,
 'replace': 88,
 '4': 89,
 're

In [ ]:
pip install pydot

In [15]:
print(dataX.shape)
print(dataY.shape)
print(len(np.unique(dataY)))
# Lets build model
vocab_size_to_predict = len(tokenizer.word_index)+1

model = Sequential()
model.add(Embedding(input_dim=vocab_size_to_predict, output_dim=32, mask_zero=True))
model.add(LSTM(256, input_shape=(dataX.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(units = vocab_size_to_predict, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True)

(130369, 10)
(130369,)
623


2022-10-16 15:50:14.287805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [16]:
## 9. Train and save the best model on Data here
filepath = "LSTM_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, 
                             save_best_only=True, mode='min')
history = model.fit(dataX, dataY, epochs=20, batch_size=64, callbacks=[checkpoint])

Epoch 1/20
2037/2038 [============================>.] - ETA: 0s - loss: 2.5602 - accuracy: 0.4267
Epoch 1: loss improved from inf to 2.56016, saving model to LSTM_model.hdf5
2038/2038 [==============================] - 62s 30ms/step - loss: 2.5602 - accuracy: 0.4267
Epoch 2/20
2036/2038 [============================>.] - ETA: 0s - loss: 1.6996 - accuracy: 0.5889
Epoch 2: loss improved from 2.56016 to 1.69965, saving model to LSTM_model.hdf5
2038/2038 [==============================] - 61s 30ms/step - loss: 1.6997 - accuracy: 0.5889
Epoch 3/20
2037/2038 [============================>.] - ETA: 0s - loss: 1.4709 - accuracy: 0.6341
Epoch 3: loss improved from 1.69965 to 1.47089, saving model to LSTM_model.hdf5
2038/2038 [==============================] - 61s 30ms/step - loss: 1.4709 - accuracy: 0.6341
Epoch 4/20
2037/2038 [============================>.] - ETA: 0s - loss: 1.3518 - accuracy: 0.6579
Epoch 4: loss improved from 1.47089 to 1.35181, saving model to LSTM_model.hdf5
2038/2038 [==

In [17]:
with open('tokenizer.json') as f:
  json_string = json.load(f)

tokenizer= keras.preprocessing.text.tokenizer_from_json(
    json_string
)

print(len(tokenizer.word_index))
tokenizer.word_index

1048


{'<UNK>': 1,
 '\n': 2,
 '(': 3,
 ')': 4,
 '=': 5,
 ':': 6,
 ',': 7,
 '.': 8,
 '1': 9,
 '_': 10,
 'a': 11,
 '"': 12,
 '[': 13,
 ']': 14,
 'x': 15,
 '0': 16,
 'i': 17,
 'print': 18,
 'input': 19,
 '*': 20,
 'y': 21,
 'n': 22,
 'int': 23,
 '2': 24,
 "'": 25,
 'if': 26,
 's': 27,
 'in': 28,
 'for': 29,
 '-': 30,
 'b': 31,
 'w': 32,
 'h': 33,
 'ans': 34,
 '+': 35,
 'split': 36,
 '/': 37,
 'r': 38,
 'else': 39,
 'range': 40,
 'max': 41,
 'map': 42,
 '#': 43,
 '9': 44,
 'list': 45,
 '%': 46,
 '\t': 47,
 'import': 48,
 'math': 49,
 'len': 50,
 'l': 51,
 'str': 52,
 'pi': 53,
 'def': 54,
 'append': 55,
 'and': 56,
 'return': 57,
 'elif': 58,
 'sys': 59,
 '>': 60,
 'c': 61,
 '3': 62,
 '5': 63,
 '{': 64,
 '}': 65,
 'f': 66,
 '!': 67,
 'm': 68,
 'gcd': 69,
 'while': 70,
 '<': 71,
 '6': 72,
 'min': 73,
 'float': 74,
 'num': 75,
 'main': 76,
 'join': 77,
 "''": 78,
 '8': 79,
 'stdin': 80,
 'd': 81,
 'format': 82,
 't': 83,
 'true': 84,
 '?': 85,
 'or': 86,
 'from': 87,
 'replace': 88,
 '4': 89,
 're

In [ ]:
tokenizer.word_docs

In [ ]:
# Above can help with large scale traing but lets focus on one file for now
dataxy = pd.read_csv('dataxy_10_1_1.csv', header = 0)
display(dataxy)
dataX = dataxy.iloc[:, 0:10].values
dataY = dataxy.iloc[:, 10].values

print(dataX)
print(dataY)

In [ ]:
# Rerun with mini dataset
print(dataX.shape)
print(dataY.shape)
print(len(np.unique(dataY)))
# Lets build model
vocab_size_to_predict = len(tokenizer.word_index)+1

model1 = Sequential()
model1.add(Embedding(input_dim=vocab_size_to_predict, output_dim=64, mask_zero=True))
model1.add(LSTM(256, input_shape=(dataX.shape[1], 1)))
model1.add(Dropout(0.01))
model1.add(Dense(1028))
model1.add(Dropout(0.01))
model1.add(Dense(units = vocab_size_to_predict, activation='softmax'))
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
tf.keras.utils.plot_model(model1, show_shapes=True, show_layer_activations=True)

In [ ]:
## 9. Train and save the best model on Mini Data here
filepath = "LSTM_model_mini_18Apr2022.hdf5"
checkpoint1 = ModelCheckpoint(filepath, monitor='loss', verbose=1, 
                             save_best_only=True, mode='min')
history1 = model1.fit(dataX, dataY, epochs=40, batch_size=256, callbacks=[checkpoint1])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history1.history['accuracy'])
# plt.plot(history1.history['loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size_to_predict, output_dim=64, mask_zero=True))
model2.add(SimpleRNN(256, input_shape=(dataX.shape[1], 1)))
model1.add(Dropout(0.01))
model1.add(Dense(1028))
model1.add(Dropout(0.01))
model2.add(Dense(units = vocab_size_to_predict, activation='softmax'))
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
tf.keras.utils.plot_model(model2, show_shapes=True, show_layer_activations=True)

In [ ]:
filepath = "RNN_model_mini_18Apr2022.hdf5"
checkpoint2 = ModelCheckpoint(filepath, monitor='loss', verbose=1, 
                             save_best_only=True, mode='min')
history2 = model2.fit(dataX, dataY, epochs=40, batch_size=256, callbacks=[checkpoint2])

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history1.history['accuracy'])
plt.title('Models accuracy')
plt.legend(['Accuracy RNN', 'Accuracy LSTM'], loc='upper left')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
pip install keras

In [ ]:
model1 = keras.models.load_model("LSTM_model.hdf5")

In [ ]:
y_p_lstm = model1.predict(dataX)
y_p_lstm = [np.argmax(y_softmax) for y_softmax in y_p_lstm]